Miscellaneous
=============

**Outline**:
1. The Python standard library
2. Formatting 
3. Duck typing and protocol
4. Enum and literal
5. Tools
6. Questions
7. Exercise
8. Closing words

## 1. The Python standard library

Python comes with many interesting modules. Here is a selection:
- `os`/`sys`: system-specific parameters and functions;
- `datetime`: manipulation of date and time (limited support for timezones);
- `logging`: built-in logging facility;
- `pathlib`: path, file OOP manipulation interface;
- `collections`: base classes for collections and a few useful concrete ones;
- `itertools`: tools related to iteration;
- `functools`: functional programming tools (decorators, partial, reduce, etc.);
- `argparse`: built-in CLI argument parsings;
- `math`/`random`: you have guessed it;
- `re`: regular expression;
- `pickle`: serialization library.


### Defaultdict

In a `defaultdict`, all values are of the same type. If you search for a key that does not exists, a new default becomes attached to that key.

In [2]:
from collections import defaultdict

data = "ABCABABCB"

histogram = defaultdict(int)

for x in data:
    histogram[x] += 1

histogram

defaultdict(int, {'A': 3, 'B': 4, 'C': 2})

In [4]:
from collections import defaultdict

# load balacing
ppl = [
    "Alice", "Bob", "Claire", "David", "Emma", 
    "Felix", "Grace", "Henry", "Isabelle", "Jack"
]

assignment = defaultdict(list) 
for p in ppl:
    assignment[hash(p) % 3].append(p)

assignment

defaultdict(list,
            {2: ['Alice', 'Emma', 'Grace', 'Isabelle'],
             0: ['Bob', 'Claire', 'Henry', 'Jack'],
             1: ['David', 'Felix']})

### Dispatching (overloading)

In a language like Java, you can have several methods with the same name and different inputs. The proper method is invoked based on the types of the arguments. In Python (which is weakly typed) you do not have this functionality out-of-the-box. The `functools` module provides a dispatching tool that might achieve the same result in some circumstances.

In [6]:
from functools import singledispatchmethod

class Negator:
    @singledispatchmethod
    def neg(self, arg):
        raise NotImplementedError("Cannot negate a")

    @neg.register
    def _(self, arg: int):
        return -arg

    @neg.register
    def _(self, arg: bool):
        return not arg

print(Negator().neg(2))
print(Negator().neg(True))

-2
False


### Itertools

Itertools provides many utilities to create iterables like cycling over a sequence, batching data, creating the cartesian product of iterables, creating an iterable of permutations or combinations, etc.

In [7]:
from itertools import permutations

list(permutations("ABC"))

[('A', 'B', 'C'),
 ('A', 'C', 'B'),
 ('B', 'A', 'C'),
 ('B', 'C', 'A'),
 ('C', 'A', 'B'),
 ('C', 'B', 'A')]

## 2. Formatting 

In Python 3.9, formatting string is done via the f-strings. Within the substitution expression of the f-string, you can give formatting options:

In [15]:
s = "blabla"
print(f"'{s:10}'")  # Make sure the length is at least 10
print(f"'{s:>10}'") # same + right aligned
print(f"'{s:^10}'") # same but centered
print(f"{0.123456789:.1f}")  # print as float (f) with 1 decimal (.1)
print(f"{0.123456789:e}")  # print in scientific notation
print(f"{'this is the repr'!r}")
print(f"{'this is the str'}")

'blabla    '
'    blabla'
'  blabla  '
0.1
1.234568e-01
'this is the repr'
this is the str


See https://docs.python.org/3/library/string.html for more.

:skull: It is possible to define formatting options for a custom class by overwriting the `__format__` method.

## 3. Duck typing and protocol

> "If it looks like a duck, swims like a duck, and quacks like a duck, then it probably is a duck."

Python has been built under the duck typing framework (aka ask forgiveness, not permission). Consider the following example:

In [1]:
class Duck:
    def quack(self):
        print("Quack, quack!")
    
class Person:
    def quack(self):
        print("Kwak, kwak")
    

def quack_it_python(duck):
    try:
        duck.quack()
    except AttributeError:
        print("Cannot quack")

def quack_it_oop(duck):
    if isinstance(duck, Duck):
        duck.quack()
    else:
        print("Cannot quack")

quack_it_python(Duck())
quack_it_python(Person())
quack_it_oop(Duck())
quack_it_oop(Person())

Quack, quack!
Kwak, kwak
Quack, quack!
Cannot quack


Duck typing does not blend well with typing, however. You can work around that thanks to the `Protocol` protocol. You can think of it an *interface* you declare afterwards:

In [3]:
from typing import Protocol

class Quacker(Protocol):
    def quack(self):
        pass

def quake_it(quacker: Quacker):
    # By typing, you move the need to ensure the proper use 
    # to the user of the function
    quacker.quack()


In [6]:
from typing import Protocol, runtime_checkable

@runtime_checkable  # Need to enable runtime checking for isinstance/issubclass
class Quacker(Protocol):
    def quack(self):
        pass

isinstance(Duck(), Quacker)

True

## 4. Enum and literals

Often you need to restrict the number of modalities of a variable to a certain range. There many ways to handle this situation in Python, and two good ones: enum and literals.

In [14]:
from enum import Enum

class RAG(Enum):
    RED = "red"
    AMBER = "amber"
    GREEN = "green"



def gimme_rag(color: RAG):
    print(f"{color} / {color.name} / {color.value}")

gimme_rag(RAG.RED)

RAG.RED / RED / red


In [12]:
from typing import Literal

def gimme_rag(color: Literal["red", "amber", "green"]):
    print(color)

gimme_rag("red")

red


:bulb: Consider using an Enum for important concepts. If the goal is just to restrict the value in a specific part of the code, the literal is way less hassle.

## 5. Tools

Although the focus of the training was on the design and code aspects, ensuring a successful and lasting project is also about good tooling:
- code versioning (git + repository);
- testing (eg. pytest);
- linting (eg. black);
- type-checking (eg. mypy);
- CI/CD;
- packet management (eg. poetry).

## 6. Questions

:question:


## 7. Exercise

Implement a game of Iterative Prisoner Dilemma.

The prisoner's dilemma is a two-player, symetric, simultaneous game where each player can take one two the following actions:
- cooperate;
- defect.

The outcome of the game is calculated as follow:
| Player 1 \ Player 2 | Cooperate  | Defect |
|---------------------|------------|--------|
| Cooperate           | 3/3        | 0/5    |
| Defect              | 5/0        | 1/1    | 

The prisonner's *dilemma* is that for any of the other player's action it is always better to defect, yet full cooperation is better than full defection.

In the iterative variant, an *a priori* unknown number of games (ie. rounds) are played in a row and the total gain decide the winner.

You must implement the game engine, at least one strategy and a report class. At the end of the game, the report should contain the following information:
- who won;
- number of rounds;
- average gain per user;
- number of time each player took each action;

Strategies can be as simple as playing always the same action, or playing a random one. Since the actions are simultaneous, a player cannot based his on the other player *for the same round*. The action can be based on *previous* rounds, however (eg. repeat the action of the other player from the round before).


## 8. Closing words 

This training was about the standard library and a few miscellaneous other things.

Note that if there are design patterns, there also exists **anti-patterns**: recipes to common problems that are counterproductive in some way (ineffective, difficult to maintain, etc.)

The trainings were about design patterns in Python, but we did not cover all the design patterns. You can easily research the remaining. As for Python, there is still plenty to explore, such as:
* data access (descriptors, MRO, slots);
* parallel computing (thread/processor pools, async, locks, GIL, etc.);
* multiple inheritance (diamond problem, mixins, MRO, composition over inheritance);
* and more.
